In [1]:
import numpy as np
import pandas as pd

import keras

from keras import preprocessing
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Input

from sklearn.preprocessing import OneHotEncoder

/Users/zfh403/.pyenv/versions/3.6.4/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_df = pd.read_csv("data/train.csv", index_col=0)
target_df = pd.read_csv("data/target.csv", index_col=0)

/Users/zfh403/.pyenv/versions/3.6.4/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
train_df.head()

,user_id,order_id,product_id_tsfm,order_number,days_since_prior_order,size
0,1,431534,1589,5,28.0,1
1,1,431534,4027,5,28.0,1
2,1,431534,4087,5,28.0,1
3,1,431534,4090,5,28.0,1
4,1,431534,4102,5,28.0,1


# Tranform into Sequence

In [4]:
X_train = train_df.groupby("user_id")["product_id_tsfm"].apply(np.hstack)

KeyboardInterrupt: 

In [ ]:
y_train = target_df.groupby("user_id")["product_id_tsfm"].apply(np.hstack)

In [ ]:
# convert to numpy
X_train = X_train.values
y_train = y_train.values

# One Hot Encode Sequence

In [ ]:
def one_hot_encode(X_train, n_products):
    return np.eye(n_products)[X_train]

In [ ]:
X_train[2]

In [ ]:
n_products = len(train_df["product_id_tsfm"].unique())

In [ ]:
example = one_hot_encode(X_train[2], n_products)

In [ ]:
example.shape

In [ ]:
len(X_train[2])

In [ ]:
X_train.shape

# Training Data Generator

In [ ]:


g = one_hot_generator()
        
def train_generator(BATCH_SIZE=10):
    x_batch = []
    y_batch = []
    
    i = 0
    for new_x, new_y in g:
        if i > BATCH_SIZE:
            break
        x_batch.append(new_x)
        y_batch.append(new_y)
        i += 1
    yield (np.array(x_batch), np.array(y_batch))

In [ ]:
x, y = next(iter(one_hot_generator()))

In [73]:
np.array(x).shape

(1, 50, 36055)

In [49]:
x, y = next(iter(train_generator()))

In [62]:
x.shape

(11,)

In [63]:
x

array([array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]),
       array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]),
       array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]),
       array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]),
    

In [53]:
x.reshape(x.shape).shape

(1, 11)

# Model 

In [127]:
def one_hot_generator():
    for x, y in zip(X_train, y_train):
        x_one_hot = one_hot_encode(x, n_products)
        y_one_hot = one_hot_encode(y, n_products)
        yield (np.array([x_one_hot]), y_one_hot)
        
# TODO: try grouping these into batches (of say 100)

In [128]:
next(iter(one_hot_generator()))

(array([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]]),
 array([[0., 0., 0., ..., 0., 0., 0.]]))

In [129]:
model = Sequential()

model.add(LSTM(256, input_shape=(None,n_products)))
model.add(Dropout(0.5))
model.add(Dense(n_products, activation='softmax'))


In [130]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
model.fit_generator(one_hot_generator(), steps_per_epoch=1000, epochs=5)

Epoch 1/5
   1/1000 [..............................] - ETA: 1:58:42 - loss: 10.4931 - acc: 0.0000e+00